# 1. Vector Space Model of Word Meaning

The goal of this problem set is to make you familiar with vector space model of word meaning. You may reuse some of functions you coded in Assignment 1. 

### Warning: This assignment may take substantial time to run if you are not optimizing your code. Make sure you have plenty of time to run if you are new to programming.

Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with current events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not. 



Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


When you run the `ls` command below, you should see the files in the tweets folder.

> Indented block






In [4]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
covid-tweets-2020-08-10-2020-08-21.trigrams.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


In [5]:
# let's read tweets. These tweets are already tokenized and cleaned (Assignment 1)
tweets = open("/content/drive/My Drive/tweets/covid-tweets-2020-08-10-2020-08-21.tokenized.txt", "r").read().split("\n")
tweets = [tweet.split() for tweet in tweets]

print(len(tweets))


312878


## Problem 1.1: Word space model

Compute the most important context words of `ventilator`. Use Pointwise Mutual Information (PMI) to rank the context words (Refer to Lecture 4).

We define context as up to 3 words to the left and 3 words to the right. Ignore stop words and words that do not start with [a-z#]. Also ignore words that are not in the top 1000 frequent words.

These context words define the dimensions of the vector space model. Represent each word as a vector (dictionary/counter) of context words with PMI as the importance of the context word. Print the top 20 context words for each.

This is the sample output I got for `ventilator`. Your numbers need not match mine but the ranked list should look close to what I have.

```
[('put', 18.280538283196606), ('wearing', 17.587373569812726), ('even', 17.58651933524197), ('like', 17.402738298715878), ('covid', 17.172590097063086), ('patients', 16.894419647496004), ('use', 16.894298589380956), ('die', 16.89426559608771), ('days', 16.89415252713107), ('needed', 16.489137134110106), ('month', 16.48907033839664), ('weeks', 16.488913820220848), ('away', 16.48879303327717), ('week', 16.488739054051933), ('person', 16.488678720881293), ('good', 16.488160838026904), ('deaths', 16.487822204799755), ('go', 16.487564042558112), ('would', 16.48707075078768), ('one', 16.48706217686235), ('get', 16.486565870239033)]
```



Let's first load stop words.

In [6]:
stop_words = set()
def load_stop_words():
  words = open("/content/drive/My Drive/tweets/stop_words.txt", "r").read().split("\n")
  for word in words:
    stop_words.add(word.strip())

load_stop_words()
print(len(stop_words))
print(stop_words)

134
{'where', 'to', 'above', 'their', "'ve", 'there', 'other', 'own', 'doing', 'up', 'both', 'i', 'do', 'each', 'a', 'this', 'most', 'his', 'as', 'will', 'amp', 'yours', 'was', 'ours', 'into', 'yourselves', 'more', 'should', 'what', 'at', 'you', 'then', 'with', 'been', "n't", 'how', 'these', 'him', 'again', 'just', 'below', 'don', 'herself', 'were', 'we', 'or', 'your', 'which', 'they', 'on', 'too', 'those', 'for', 'few', 'be', 'in', 'that', 'under', 'by', 'same', 'me', 'does', 'here', 'nor', 'being', 'himself', "'ld", 'who', 'once', '@USER', 'through', "'ll", 't', 'and', 'out', 'all', 'against', 'over', 'while', 'are', 'am', 'from', 'itself', 'theirs', 'our', 'he', 'is', 'can', 'them', 'about', 'ourselves', 'hers', 'if', 'not', 'because', 'themselves', 'but', 'it', 'yourself', 'after', 'the', 'have', 'myself', 'between', 'an', 'down', 'until', 'now', 'so', 'URL', 'why', 'some', 'during', 'off', 'only', 'further', 's', 'any', 'her', 'when', 'she', 'had', 'did', 'of', 'such', 'whom', 'my



```
# This is formatted as code
```

Let's build the word vectors

In [6]:
# THIS CODE IS SHITAAAAZZZZZ
# help here: https://www.kaggle.com/gabrielaltay/word-vectors-from-pmi-matrix
from collections import Counter
import itertools

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import linalg 
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

headlines = [[token for token in tweet if token not in stop_words and (re.match(r"[a-z#]+", token[0]))] for tweet in tweets]

unigram_counts = Counter()
skipgram_counts = Counter()
x2i, i2x = {}, {}
pmi_samples = Counter()

unigram_counts = Counter()
for ii, headline in enumerate(headlines):
    if ii % 200000 == 0:
        print(f'finished {ii/len(headlines):.2%} of headlines')
    for token in headline:
        unigram_counts[token] += 1

tok2indx = {tok: indx for indx, tok in enumerate(unigram_counts.keys())}
indx2tok = {indx: tok for tok,indx in tok2indx.items()}
print('done')
print('vocabulary size: {}'.format(len(unigram_counts)))
print('most common: {}'.format(unigram_counts.most_common(10)))





back_window = 2
front_window = 2
skipgram_counts = Counter()
for iheadline, headline in enumerate(headlines):
    tokens = [tok2indx[tok] for tok in headline]
    for ii_word, word in enumerate(tokens):
        ii_context_min = max(0, ii_word - back_window)
        ii_context_max = min(len(headline) - 1, ii_word + front_window)
        ii_contexts = [
            ii for ii in range(ii_context_min, ii_context_max + 1) 
            if ii != ii_word]
        for ii_context in ii_contexts:
            skipgram = (tokens[ii_word], tokens[ii_context])
            skipgram_counts[skipgram] += 1    
    if iheadline % 200000 == 0:
        print(f'finished {iheadline/len(headlines):.2%} of headlines')
        
print('done')
print('number of skipgrams: {}'.format(len(skipgram_counts)))
most_common = [
    (indx2tok[sg[0][0]], indx2tok[sg[0][1]], sg[1]) 
    for sg in skipgram_counts.most_common(10)]
print('most common: {}'.format(most_common))



row_indxs = []
col_indxs = []
dat_values = []
ii = 0
for (tok1, tok2), sg_count in skipgram_counts.items():
    ii += 1
    if ii % 1000000 == 0:
        print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')    
    row_indxs.append(tok1)
    col_indxs.append(tok2)
    dat_values.append(sg_count)
wwcnt_mat = sparse.csr_matrix((dat_values, (row_indxs, col_indxs)))
print('done')


def ww_sim(word, mat, topn=100):
    """Calculate topn most similar words to word"""
    indx = tok2indx[word]
    if isinstance(mat, sparse.csr_matrix):
        v1 = mat.getrow(indx)
    else:
        v1 = mat[indx:indx+1, :]
    sims = cosine_similarity(mat, v1).flatten()
    sindxs = np.argsort(-sims)
    sim_word_scores = [(indx2tok[sindx], sims[sindx]) for sindx in sindxs[0:topn]]
    return sim_word_scores

wwcnt_norm_mat = normalize(wwcnt_mat, norm='l2', axis=1)
row = wwcnt_mat.getrow(10).toarray().flatten()
print(np.sqrt((row*row).sum()))

row = wwcnt_norm_mat.getrow(10).toarray().flatten()
print(np.sqrt((row*row).sum()))








num_skipgrams = wwcnt_mat.sum()
assert(sum(skipgram_counts.values())==num_skipgrams)

# for creating sparce matrices
row_indxs = []
col_indxs = []

pmi_dat_values = []    # pointwise mutual information
ppmi_dat_values = []   # positive pointwise mutial information
spmi_dat_values = []   # smoothed pointwise mutual information
sppmi_dat_values = []  # smoothed positive pointwise mutual information

# reusable quantities

# sum_over_rows[ii] = sum_over_words[ii] = wwcnt_mat.getcol(ii).sum()
sum_over_words = np.array(wwcnt_mat.sum(axis=0)).flatten()
# sum_over_cols[ii] = sum_over_contexts[ii] = wwcnt_mat.getrow(ii).sum()
sum_over_contexts = np.array(wwcnt_mat.sum(axis=1)).flatten()

# smoothing
alpha = 0.75
sum_over_words_alpha = sum_over_words**alpha
nca_denom = np.sum(sum_over_words_alpha)



ii = 0
for (tok_word, tok_context), sg_count in skipgram_counts.items():

    ii += 1
    if ii % 1000000 == 0:
        print(f'finished {ii/len(skipgram_counts):.2%} of skipgrams')

    # here we have the following correspondance with Levy, Goldberg, Dagan
    #========================================================================
    #   num_skipgrams = |D|
    #   nwc = sg_count = #(w,c)
    #   Pwc = nwc / num_skipgrams = #(w,c) / |D|
    #   nw = sum_over_cols[tok_word]    = sum_over_contexts[tok_word] = #(w)
    #   Pw = nw / num_skipgrams = #(w) / |D|
    #   nc = sum_over_rows[tok_context] = sum_over_words[tok_context] = #(c)
    #   Pc = nc / num_skipgrams = #(c) / |D|
    #
    #   nca = sum_over_rows[tok_context]^alpha = sum_over_words[tok_context]^alpha = #(c)^alpha
    #   nca_denom = sum_{tok_content}( sum_over_words[tok_content]^alpha )
    
    nwc = sg_count
    Pwc = nwc / num_skipgrams
    nw = sum_over_contexts[tok_word]
    Pw = nw / num_skipgrams
    nc = sum_over_words[tok_context]
    Pc = nc / num_skipgrams
    
    nca = sum_over_words_alpha[tok_context]
    Pca = nca / nca_denom
    
    # note 
    # pmi = log {#(w,c) |D| / [#(w) #(c)]} 
    #     = log {nwc * num_skipgrams / [nw nc]}
    #     = log {P(w,c) / [P(w) P(c)]} 
    #     = log {Pwc / [Pw Pc]}
    pmi = np.log2(Pwc/(Pw*Pc))   
    ppmi = max(pmi, 0)
    spmi = np.log2(Pwc/(Pw*Pca))
    sppmi = max(spmi, 0)
    
    row_indxs.append(tok_word)
    col_indxs.append(tok_context)
    pmi_dat_values.append(pmi)
    ppmi_dat_values.append(ppmi)
    spmi_dat_values.append(spmi)
    sppmi_dat_values.append(sppmi)
        
pmi_mat = sparse.csr_matrix((pmi_dat_values, (row_indxs, col_indxs)))
ppmi_mat = sparse.csr_matrix((ppmi_dat_values, (row_indxs, col_indxs)))
spmi_mat = sparse.csr_matrix((spmi_dat_values, (row_indxs, col_indxs)))
sppmi_mat = sparse.csr_matrix((sppmi_dat_values, (row_indxs, col_indxs)))

print('done')


NameError: ignored

In [ ]:
# THIS CODE IS SHITAAAAZZZZZ
pmi_use = ppmi_mat
embedding_size = 50
uu, ss, vv = linalg.svds(pmi_use, embedding_size) 

unorm = uu / np.sqrt(np.sum(uu*uu, axis=1, keepdims=True))
vnorm = vv / np.sqrt(np.sum(vv*vv, axis=0, keepdims=True))
#word_vecs = unorm
#word_vecs = vnorm.T
word_vecs = uu + vv.T
word_vecs_norm = word_vecs / np.sqrt(np.sum(word_vecs*word_vecs, axis=1, keepdims=True))


In [ ]:
ww_sim('ventilator', ppmi_mat)

In [ ]:
# THIS CODE IS SHITAAAAZZZZZ
# help here: https://www.kaggle.com/gabrielaltay/word-vectors-from-pmi-matrix
from collections import Counter
from itertools import combinations
from math import log
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pformat
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, norm
from nltk.util import skipgrams
from string import punctuation
from scipy import sparse
from scipy.sparse import linalg
import re
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

tweets_trimmed = [[token for token in tweet if token not in stop_words and (re.match(r"[a-z#]+", token[0]))] for tweet in tweets]

unigram_counts = Counter()
skipgram_counts = Counter()
x2i, i2x = {}, {}
pmi_samples = Counter()



def compute_unigrams():
  global unigram_counts
  for idx, tweet in enumerate(tweets_trimmed):
    for token in tweet:
      unigram_counts[token] += 1
    if idx % 50000 == 0:
        print(f'finished {idx/len(tweets):.2%} of tweets')

  # # Remove all unigrams in in top 1000
  # most_common_unigram = unigram_counts.most_common(1000)
  # for k,v in list(unigram_counts.items()):
  #   if (k,v) not in most_common_unigram:
  #     del unigram_counts[k]
  # print('Trimmed vocab size should be 1000: {}'.format(len(unigram_counts)))

def compute_skip_grams():
  global skipgram_counts
  for idx, tweet in enumerate(tweets_trimmed):
      # 3 words left and right
      skip_gram = list(skipgrams(tweet, 2, 2))
      for context in skip_gram:
        skipgram_counts[context] += 1
      if idx % 50000 == 0:
        print(f'finished {idx/len(tweets_trimmed):.2%} of tweets')

  # Remove infrequent bigrams (not in unigrams)
  # for x, y in list(skipgram_counts.keys()):
  #   if x not in unigram_counts or y not in unigram_counts:
  #     del skipgram_counts[(x,y)]

  print('Skipgram size: {}'.format(len(skipgram_counts)))
  print('Most common skipgrams: {}'.format(skipgram_counts.most_common(10)))


def build_unigram_index_lookup():
  global x2i
  global i2x
  for i, x in enumerate(unigram_counts.keys()):
    x2i[x] = i
    i2x[i] = x

    
def PMI(word1, word2):
  # You have to store frequencies of individual words and (word, context word) 
  # pairs to compute this. You can compute them beforehand in order to avoid 
  # counting every time when this function is called.
  # Write your code
  global skipgram_counts
  global unigram_counts
  # a = (sg_count / sxy)
  # b = (unigram_counts[word1] / sx)
  # c = (unigram_counts[word2] / sx)
  nxy = skipgram_counts[(word1, word2)]
  N = len(tweets_trimmed)
  nx = unigram_counts[word1]
  ny = unigram_counts[word2]
  pmi = log((nxy * N) / (nx * ny))
  return pmi

def build_word_vector(word):
  # Write your code
  global skipgram_counts
  global unigram_counts
  global x2i
  global i2x
  sx = sum(unigram_counts.values())
  sxy = sum(skipgram_counts.values())
  data, rows, cols = [], [], []
  pmi_sample = Counter()

  for (x, y), n in skipgram_counts.items():
    rows.append(x2i[x])
    cols.append(x2i[y])
    data.append(PMI(x,y))
    pmi_sample[(x,y)] = data[-1]

  PMI_matrix = csc_matrix((data, (rows, cols)))
  ans = Counter()
  print(pmi_sample.most_common()[:20])
  for (x,y), n in pmi_sample.items():
    if (x == word or y == word):
      ans[(x,y)] = n
  print(ans.most_common(5))

def print_top_dimensions(word_vector, n):
  # print top n dimensions sorted in the order of importance.
  ...

compute_unigrams()
print()
compute_skip_grams()
print()
build_unigram_index_lookup()

v1 = build_word_vector('ventilator')

# print_top_dimensions(v1, 20) # print top 20 dimensions along with their weights


## Problem 1.2: Compute the word similarity between words using Cosine Similarity.

Compute cosine similarity between the following pair of words: 
```
('ventilator', 'covid-19')
('ventilator', 'lockdown')
('ventilator', 'mask')
('ventilator', 'ppe')
```

Outputs of my code are:
```
('ventilator', 'covid-19') 0.17076006036635358
('ventilator', 'lockdown') ...
('ventilator', 'mask') 0.19229601085517933
('ventilator', 'ppe') ...
```


In [7]:
def cosine_similarity(vector1, vector2):
  v2 = list(vector1.values())
  v2 = list(vector2.values())

  dot_prod = np.dot(v1, v2)
  mag = LA.norm(v1) * LA.norm(v2)
  return dot_prod / mag

ventilator = build_word_vector('ventilator')
covid19 = build_word_vector('covid-19')
lockdown = build_word_vector('lockdown')
mask = build_word_vector('mask')
ppe = build_word_vector('ppe')

print(('ventilator', 'covid-19'), cosine_similarity(ventilator, covid19))
print(('ventilator', 'lockdown'), cosine_similarity(ventilator, lockdown))
print(('ventilator', 'mask'), cosine_similarity(ventilator, mask))
print(('ventilator', 'ppe'), cosine_similarity(ventilator, ppe))

NameError: ignored

## Problem 1.3: What can you tell about these words from the similarities?

1. `ventilator` when compared with `covid-19, lockdown, mask, ppe`
2. `pandemic` when compared with `covid-19, lockdown, mask, ppe`
3. `president` compared with `trump, biden`
4. `trudeau` compared with `trump, biden`



# Let's play with word2vec

First let's load word2vec. I am using [gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) but feel free to use any libraries or tools.

In [8]:
from gensim.models import KeyedVectors

EMBEDDING_FILE = '/content/drive/My Drive/tweets/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

## Problem 1.4: Compute the top 5 similar words of `ventilator` using word2vec?

---



In [ ]:
# Write your code here
word2vec.most_similar(['ventilator'] , topn = 5)

# Problem 1.5: Word analogy

If I told you the plural of `car` is `cars`, can you automatically find the plural of `hypothesis` and `man` using word2vec.

Similarly, if I told you a newborn `dog` is called `puppy`, can you automatically find what are the newborn words of `cat` and `sheep` using word2vec?


In [ ]:
# Write your code here


# 2. Topic Models 

The goal of this part is to make you familiar with topic models. You may reuse some of functions you coded for the previous assignments.

## Data Download and Setup

Let us start by downloading the news section of the Brown corpus:

In [7]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
documents = [brown.words(fileid) for fileid in brown.fileids()]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


Let us inspect some of the documents:

In [8]:
print("The news section of the Brown corpus contains {} documents.".format(len(documents)))
for i in range(3):
  document = documents[i]
  print("Document {} has {} words: {}".format(i, len(document), document))

The news section of the Brown corpus contains 500 documents.
Document 0 has 2242 words: ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]
Document 1 has 2277 words: ['Austin', ',', 'Texas', '--', 'Committee', 'approval', ...]
Document 2 has 2275 words: ['Several', 'defendants', 'in', 'the', 'Summerdale', ...]


Finally, let us download a list of stopwords for later.

In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
print(stopwords_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

## Problem 2.1: Document-Term Matrix

Create a document-term matrix with tf-idf. You should preprocess documents by: 1) lowercasing words, 2) excluding stopwords, and 3) including alphanumeric strings only.

```python
import numpy as np
def create_tfidf_matrix(documents: List[List[str]]) -> (np.array, List[str]):
  # Args:
  #   documents: list of documents, each document being a list of words.
  # Outputs:
  #   tfidf_matrix: np.array of shape (num_documents, vocabulary_size)
  #   vocabulary: a list of terms corresponding to the columns of the matrix.

tfidf_matrix, vocabulary = create_tfidf_matrix(documents)
```

How sparse is this matrix? Calculate the ratio between cells with value 0 and the total number of cells. 


In [10]:
import re
from typing import List

def preprocess(documents: List[List[str]]):
  return_docs = []
  documents_processed = [[token.lower() for token in document if token not in stop_words and token.isalnum()] for document in documents]
  for doc in documents_processed:
    return_docs.append(' '.join(doc))

  return return_docs

documents_processed = preprocess(documents)

In [11]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


def create_tfidf_matrix(documents: List[List[str]]) -> (np.array, List[str]):
  # Args:
  #   documents: list of documents, each document being a list of words.
  # Outputs:
  #   tfidf_matrix: np.array of shape (num_documents, vocabulary_size)
  #   vocabulary: a list of terms corresponding to the columns of the matrix.
  vectorizer = TfidfVectorizer()
  doc_vec = vectorizer.fit_transform(documents)
  tfidf_df = pd.DataFrame(doc_vec.toarray(), columns = vectorizer.get_feature_names())
  tfidf_matrix = tfidf_df.to_numpy()
  voc = list(tfidf_df.columns.values)
  return (tfidf_matrix, voc)

tfidf_matrix, vocabulary = create_tfidf_matrix(documents_processed)

In [12]:
# SPACITY OF MATRIX
from numpy import count_nonzero

sparsity = 1.0 - ( count_nonzero(tfidf_matrix) / float(tfidf_matrix.size) )
print("Sparcity of a matrix is {}".format(sparsity))

Sparcity of a matrix is 0.9840189393014912


## Problem 2.2: Latent Semantic Analysis

We perform LSA to obtain document embeddings `U` and term embeddings `VT`.

```python
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(tfidf_matrix, 
                              n_components=10,
                              n_iter=100,
                              random_state=42)
```

Define a function to find the 5 most relevant terms for each of the 10 latent dimensions (tip: you should make use of VT and the vocabulary).

```python
def extract_salient_words(VT: np.array, 
                  vocabulary: List[str]
                  ) -> salient_words: dict[int, List[str]]:
  # Args:
  #  VT: a numpy array of size (n_components, vocabulary_size)
  #  vocabulary: a list of words of size vocabulary_size
  # Outputs:
  #  salient_words: a dictionary with the latent dimension indices as keys and a list of its 5 most salient words as values

salient_words = extract_salient_words(VT, vocabulary)

for key, value in salient_words:
  print("Concept {}: {}".format(str(key), " ".join(value)))
```

In [13]:
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(tfidf_matrix, 
                              n_components=10,
                              n_iter=100,
                              random_state=42)

In [56]:
from typing import List, Dict

def extract_salient_words(VT: np.array, vocabulary: List[str]) -> Dict[int, List[str]]:
  # Args:
  #  VT: a numpy array of size (n_components, vocabulary_size)
  #  vocabulary: a list of words of size vocabulary_size
  # Outputs:
  #  salient_words: a dictionary with the latent dimension indices as keys and a list of its 5 most salient words as values
  salient_words: Dict[int, List[str]] = {}
  for i in range(len(VT)):
    tfidf_sorting = np.argsort(VT[i]).flatten()[::-1]
    n = 5
    voc = np.array(vocabulary)
    top_n = voc[tfidf_sorting]
    salient_words[i] = top_n[:n].tolist()
  return salient_words

salient_words = extract_salient_words(VT, vocabulary)

for key, value in salient_words.items():
  print("Concept {}: {}".format(str(key), " ".join(value)))

Concept 0: the he one said would
Concept 1: he she said back like
Concept 2: af shear temperature anode tangent
Concept 3: life human god experience literature
Concept 4: af he nations soviet united
Concept 5: af music the john kennedy
Concept 6: school church education college said
Concept 7: god church christ him jesus
Concept 8: he the students education college
Concept 9: he state court federal tax


## Problem 2.3: Document Retrieval

Given a text query, view this as a mini document, and compare it to your documents in the low-dimensional space.

First, we need to map a query into a pseudo-document embedding.
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/ed5d0397ee6b44f72f77743029d3943932118fa2" alt="Query" height="35"/>

Then, you will need to implement a function to calculate the cosine similarity between this embedded query and all the document embeddings.

Retrieve the indices of the top-3 documents with the highest cosine similarity with the following queries:


```python
query1 = ['T.', 'V.', 'Barker', 'developed', 'the', 'classification-angle', 'system']
query2 = ['imitation', 'vs.', 'formalism' 'in', 'philosophical', 'debates']
query3 = ['Krim', 'attended', 'the', 'University', 'of', 'North', 'Carolina', 'to', 'follow', 'Thomas', 'Wolfe']
```



## Problem 2.4: Document Clustering

```python
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

num_clusters = 10
km = KMeans(n_clusters=num_clusters)
document_embeddings = U * Sigma
km.fit(document_embeddings)
clusters = km.labels_.tolist()
print(clusters)
```

Let us now plot the document embeddings and their clusters:

```python
import umap
embedding = umap.UMAP(n_neighbors=100, min_dist=0.5, random_state=42).fit_transform(document_embeddings)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], c=clusters, s=20, edgecolor='none')
plt.show()
```

What are the differences you observe by using a different number of `n_components` in LSA or `n_clusters` in K-Means?

In [79]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import umap

In [80]:
num_clusters = 20
km = KMeans(n_clusters=num_clusters)
document_embeddings = U * Sigma
km.fit(document_embeddings)
clusters = km.labels_.tolist()
print(clusters)

[14, 14, 14, 5, 14, 14, 5, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 6, 6, 10, 15, 15, 15, 11, 15, 5, 7, 10, 5, 0, 15, 15, 19, 5, 6, 13, 19, 5, 5, 19, 19, 5, 19, 4, 0, 0, 5, 3, 13, 19, 14, 14, 19, 5, 19, 5, 19, 9, 9, 11, 5, 19, 19, 11, 11, 11, 11, 16, 11, 11, 11, 11, 11, 11, 11, 11, 11, 15, 19, 1, 16, 3, 3, 12, 0, 12, 12, 13, 3, 3, 16, 7, 1, 3, 16, 12, 0, 1, 1, 1, 11, 0, 6, 1, 2, 0, 1, 1, 15, 0, 1, 1, 1, 1, 1, 1, 1, 1, 11, 13, 13, 1, 13, 1, 6, 7, 6, 1, 1, 6, 1, 1, 6, 13, 8, 16, 1, 1, 16, 13, 13, 4, 0, 1, 13, 6, 19, 3, 8, 1, 0, 13, 0, 1, 8, 9, 0, 7, 1, 1, 0, 19, 7, 0, 15, 7, 1, 1, 8, 3, 15, 13, 13, 15, 16, 11, 3, 1, 19, 15, 9, 19, 9, 19, 8, 13, 11, 19, 13, 16, 19, 16, 16, 16, 8, 16, 16, 19, 11, 13, 13, 19, 9, 16, 16, 16, 16, 0, 16, 16, 13, 0, 0, 16, 0, 9, 0, 0, 0, 11, 0, 0, 16, 16, 16, 0, 16, 0, 8, 11, 8, 0, 0, 0, 0, 3, 16, 7, 0, 0, 0, 13, 19, 16, 0, 16, 0, 0, 13, 16, 16, 16, 9, 11, 16, 17, 6, 9, 19, 10, 10, 6, 10, 14, 10, 1, 2, 10, 5, 10, 1, 10, 10, 9, 9, 

In [81]:
embedding = umap.UMAP(n_neighbors=100, min_dist=0.5, random_state=42).fit_transform(document_embeddings)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], c=clusters, s=20, edgecolor='none')
plt.show()

## Problem 2.5 Latent Dirichlet Allocation

Run LDA on `documents` using `sklearn` (find the documentation at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation))

Make sure to specify `random_state=42` for replicability. 

What are the topics allocated to each word of document number 13? 

In [ ]:
print(documents[13])

In [ ]:
# Write your code here.
    
